Mohammad Mahdi Gheidi

98105976

Natural Language Processing Course - HW1 

Fall 2023 - Dr. Ehsaneddin Asgari


در این نوت‌بوک به بررسی و آنالیز داده‌های آگهی‌های دسته خودروی سواری سایت دیوار خواهیم پرداخت.

داده‌های مورد استفاده در این نوتبوک به وسیله اسکریپت نوشته شده پایتون که کنار این فایل قرار دارد، از سایت دیوار کراول شده است.

در ابتدا داده‌ها را لود می‌کنیم و داده‌های مربوط به هر شهر را در متغیر‌های مربوط به خودش قرار می‌دهیم.

سپس به وسیله ابزار هضم، به پاکسازی داده‌ها و نرمال کردنشان می‌پردازیم.

در انتها نیز به بررسی و آنالیز داده‌های مربوط به آگهی‌ها می‌پردازیم.

In [ ]:
%pip install -r requirements.txt